# Performance analysis at application-level

## Reproduce number of method calls

In [32]:
import py2neo
import pandas as pd

# establish connection to Neo4j database with performance data
graph = py2neo.Graph(bolt=True, host='localhost')

# create cypher query to get the total number of method calls
query_method_calls ="""
MATCH (method1:Method)-[calls:CALLS]->(method2:Method)
RETURN SUM(calls.weight) AS methodCalls
"""

# execute query and store result in a dataframe
df_method_calls = pd.DataFrame(graph.run(query_method_calls).data())

# print out the total number of method calls (2,409,688,701)
df_method_calls.head()

,methodCalls
0,2409688701


## Reproduce weighted dynamic dependency graph on class level

In [33]:
# create cypher query to calculate import, export, and combined coupling degrees
# derived from the weighted dynamic dependency graph on class level
query_c_w ="""
MATCH (type:Type:Kieker)
WHERE ((type)-[:DEPENDS_ON]->(:Type:Kieker) OR (:Type:Kieker)-[:DEPENDS_ON]->(type))
WITH type
OPTIONAL MATCH (type)-[out:DEPENDS_ON]->(:Type:Kieker)
WITH type, SUM(out.weight) AS import
OPTIONAL MATCH (:Type:Kieker)-[in:DEPENDS_ON]->(type)
WITH type.fqn AS class, import, SUM(in.weight) as export
RETURN class, import, export, (import+export) AS combined
ORDER BY class
"""

# execute query and store result in a dataframe
df_c_w = pd.DataFrame(graph.run(query_c_w).data())

# print out the average export coupling degree on class level (370,821)
print(round(df_c_w["export"].mean()))

370821.0


## Reproduce weighted dynamic dependency graph on package level

In [39]:
# create cypher query to calculate import, export, and combined coupling degrees
# derived from the weighted dynamic dependency graph on package level
query_p_w ="""
MATCH (package:Package:Kieker)
WHERE ((package)-[:DEPENDS_ON]->(:Package:Kieker) OR (:Package:Kieker)-[:DEPENDS_ON]->(package))
WITH package
OPTIONAL MATCH (package)-[out:DEPENDS_ON]->(:Package:Kieker)
WITH package, SUM(out.weight) AS import
OPTIONAL MATCH (:Package:Kieker)-[in:DEPENDS_ON]->(package)
WITH package.fqn AS package, import, SUM(in.weight) as export
RETURN package, import, export, (import+export) AS combined
ORDER BY package
"""

# execute query and store result in a dataframe
df_p_w = pd.DataFrame(graph.run(query_p_w).data())

# print out the average export coupling degree on package level (1,868,664)
print(round(df_p_w["export"].mean()))

1868664.0
